In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import Output, VBox
import openpyxl  # To access cell formatting

# Prompting user to upload an Excel file
print("Please upload your Excel file.")
uploaded = files.upload()

# Get uploaded file name
file_name = list(uploaded.keys())[0]

# Reading the Excel file and get the sheet names
xl = pd.ExcelFile(file_name, engine='openpyxl')
sheet_names = xl.sheet_names

# Dropdown to select the sheet
print("\nSelect Sheet:")
sheet_dropdown = widgets.Dropdown(options=sheet_names, description='Sheet:')
display(sheet_dropdown)

# output widgets to display data preview and plot
output_preview = Output()
output_plot = Output()

# Identifying percentage-formatted columns
def get_percentage_columns(file_name, sheet_name):
    wb = openpyxl.load_workbook(file_name, data_only=True)
    ws = wb[sheet_name]
    percentage_columns = set()
    for column_cells in ws.iter_cols(min_row=2):
        for cell in column_cells:
            number_format = cell.number_format
            if '%' in number_format:
                col_letter = cell.column_letter
                percentage_columns.add(col_letter)
                break  # Stop after finding one cell in the column
    # Mapping column letters to column names
    header_row = next(ws.iter_rows(min_row=1, max_row=1))
    col_letter_to_name = {cell.column_letter: str(cell.value) for cell in header_row if cell.value}
    percentage_column_names = [col_letter_to_name[col_letter] for col_letter in percentage_columns if col_letter in col_letter_to_name]
    return percentage_column_names

# Load and display data of the selected sheet
def load_sheet(sheet_name):
    # Clear previous outputs to avoid clutter
    with output_preview:
        clear_output()
        global df, percentage_columns
        # Read the sheet into DataFrame
        df = pd.read_excel(file_name, sheet_name=sheet_name, engine='openpyxl')

        # Identify percentage-formatted columns
        percentage_columns = get_percentage_columns(file_name, sheet_name)

        # Convert percentage-formatted columns
        if percentage_columns:
            for col in percentage_columns:
                if col in df.columns:
                    df[col] = df[col] * 100  # Convert decimal to percentage
                    df[col] = df[col].round(2)  # Rounding to 2 decimal places

        # Preview of the data contained in the DataFrame
        print("\nData Preview:")
        display(df.head())

        # Identifying numeric columns for Y-axis
        numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

        # Update dropdown options for columns
        column_options = df.columns.tolist()
        x_dropdown.options = column_options
        identifier_dropdown.options = column_options
        y_dropdown.options = numeric_cols  # Only numeric columns for Y-axis

        # Reset dropdown values
        x_dropdown.value = None
        y_dropdown.value = None
        identifier_dropdown.value = None
        chart_type_dropdown.value = 'Bar Chart'  # Default to 'Bar Chart'

        # Clear previous plot
        with output_plot:
            clear_output()

# Update plot based on selected columns and chart type
def update_plot(change=None):
    with output_plot:
        clear_output()
        plt.close('all')
        if x_dropdown.value and y_dropdown.value and identifier_dropdown.value:
            plot_columns(x_dropdown.value, y_dropdown.value, identifier_dropdown.value, chart_type_dropdown.value)

# Plot columns with identifier and selected chart type
def plot_columns(x_column, y_column, identifier_column, chart_type):
    # New figure
    plt.figure(figsize=(12,7))

    # Copy the data to avoid modifying the original DataFrame
    plot_df = df.copy()

    # Exclude rows with empty or NaN values in x_column, y_column, and identifier_column
    plot_df = plot_df.dropna(subset=[x_column, y_column, identifier_column])

    # Exclude rows with non-numeric values in y_column (shouldn't be necessary, but extra safety)
    plot_df = plot_df[pd.to_numeric(plot_df[y_column], errors='coerce').notnull()].copy()
    plot_df[y_column] = pd.to_numeric(plot_df[y_column], errors='coerce')

    # Check if plot_df is empty after data cleaning
    if plot_df.empty:
        print("No data available to plot after cleaning. Please check your data or column selections.")
        return  # Exit the function early since there's nothing to plot

    # Handle 'Pool' column if it exists;; In case of a column titled pool containing multiple types like 'Pool-1'. 'Pool-2' etc.
    pool_exists = 'Pool' in plot_df.columns
    if pool_exists:
        # Clean 'Pool' column by removing rows with null values or 'N/A' values
        plot_df = plot_df[plot_df['Pool'].notnull()].copy()
        plot_df = plot_df[~plot_df['Pool'].astype(str).str.upper().isin(['N/A', 'NA'])].copy()
        if plot_df['Pool'].empty:
            pool_exists = False

    # Map categorical x_column to strings
    plot_df[x_column] = plot_df[x_column].astype(str)

    # Determine if x_column is numeric or categorical
    x_is_numeric = pd.api.types.is_numeric_dtype(plot_df[x_column])

    if pool_exists:
        # Plotting when 'Pool' column exists
        plot_df = plot_df.sort_values(by=['Pool', x_column])
        pools = plot_df['Pool'].unique()
        colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown']
        plot_df['Pool_X'] = plot_df['Pool'] + ' - ' + plot_df[x_column]
        x_categories = plot_df['Pool_X'].unique()
        x_mapping = {category: position for position, category in enumerate(x_categories)}
        plot_df['x_mapped'] = plot_df['Pool_X'].map(x_mapping)
        y_values = plot_df[y_column]

        # Plot bars grouped by Pool
        for i, pool in enumerate(pools):
            pool_df = plot_df[plot_df['Pool'] == pool]
            if pool_df.empty:
                continue
            pool_x_values = pool_df['x_mapped']
            pool_y_values = pool_df[y_column]
            label = f"{pool}"
            color = colors[i % len(colors)]
            if chart_type == 'Bar Chart':
                plt.bar(pool_x_values, pool_y_values, label=label, color=color)
            elif chart_type == 'Line Plot':
                plt.plot(pool_x_values, pool_y_values, marker='o', linestyle='-', label=label, color=color)
            elif chart_type == 'Scatter Plot':
                plt.scatter(pool_x_values, pool_y_values, label=label, color=color)

        plt.xticks(ticks=range(len(x_categories)), labels=x_categories, rotation=90)
        plt.legend(title='Pool')

        # Annotate each point with the identifier
        for idx, row in plot_df.iterrows():
            x_point = row['x_mapped']
            y_point = row[y_column]
            identifier_text = str(row[identifier_column])
            plt.annotate(identifier_text, (x_point, y_point),
                         xytext=(0,5), textcoords='offset points', ha='center', fontsize=9)

        # Annotate min and max points for each pool
        for pool in pools:
            pool_df = plot_df[plot_df['Pool'] == pool]
            if not pool_df.empty:
                min_idx = pool_df[y_column].idxmin()
                max_idx = pool_df[y_column].idxmax()

                # Min point
                annotate_point(pool_df, min_idx, 'Min', 'red', x_column, y_column)
                # Max point
                annotate_point(pool_df, max_idx, 'Max', 'green', x_column, y_column)
    else:
        # Plotting when 'Pool' column does not exist
        plot_df = plot_df.sort_values(by=x_column)
        x_categories = plot_df[x_column].unique()
        x_mapping = {category: position for position, category in enumerate(x_categories)}
        plot_df['x_mapped'] = plot_df[x_column].map(x_mapping)
        y_values = plot_df[y_column]
        x_positions = plot_df['x_mapped']

        if chart_type == 'Bar Chart':
            plt.bar(x_positions, y_values, color='skyblue')
        elif chart_type == 'Line Plot':
            plt.plot(x_positions, y_values, marker='o', linestyle='-', color='skyblue')
        elif chart_type == 'Scatter Plot':
            plt.scatter(x_positions, y_values, color='skyblue')

        plt.xticks(ticks=range(len(x_categories)), labels=x_categories, rotation=45)

        # Annotate each point with the identifier
        for idx, row in plot_df.iterrows():
            x_point = row['x_mapped']
            y_point = row[y_column]
            identifier_text = str(row[identifier_column])
            plt.annotate(identifier_text, (x_point, y_point),
                         xytext=(0,5), textcoords='offset points', ha='center', fontsize=9)

        # Annotate overall min and max
        min_idx = plot_df[y_column].idxmin()
        max_idx = plot_df[y_column].idxmax()

        # Min point
        annotate_point(plot_df, min_idx, 'Min', 'red', x_column, y_column)
        # Max point
        annotate_point(plot_df, max_idx, 'Max', 'green', x_column, y_column)

    plt.title(f"{y_column} vs {x_column}", fontsize=16)
    plt.xlabel(x_column, fontsize=12)
    plt.ylabel(y_column, fontsize=12)

    # Format axis labels if necessary
    if y_column in percentage_columns:
        plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.2f}%'.format(y)))
    if x_column in percentage_columns:
        plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.2f}%'.format(x)))

    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

def annotate_point(plot_df, idx, label, color, x_column, y_column):
    x_point = plot_df['x_mapped'].loc[idx]
    y_point = plot_df[y_column].loc[idx]

    # Get the actual x and y values for annotation
    x_value = plot_df[x_column].loc[idx]
    y_value = plot_df[y_column].loc[idx]

    # Format x and y values for display
    if y_column in percentage_columns:
        y_display = f"{y_value:.2f}%"
    else:
        y_display = str(y_value)

    if x_column in percentage_columns:
        x_display = f"{x_value:.2f}%"
    else:
        x_display = str(x_value)

    # Annotation text
    annotation_text = f"{label}: {x_display}, {y_display}" ###Output in the bubble###

    # Plot the point with a distinctive marker
    plt.scatter(x_point, y_point, color=color, s=120, marker='+', zorder=5)

    # Annotate the point with a text box
    plt.annotate(
        annotation_text,
        xy=(x_point, y_point),
        xytext=(10, 10), textcoords='offset points',
        ha='left', va='bottom',
        fontsize=10,
        bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.8),
        arrowprops=dict(facecolor=color, arrowstyle='->')
    )

# Function to update Identifier when Y-axis changes
def on_y_axis_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        new_value = change['new']
        if new_value is not None and new_value in identifier_dropdown.options:
            identifier_dropdown.value = new_value

# Dropdown widgets for columns and chart type
x_dropdown = widgets.Dropdown(description='X-axis:')
y_dropdown = widgets.Dropdown(description='Y-axis (Numeric Only):')  # Updated description
identifier_dropdown = widgets.Dropdown(description='Identifier:')
chart_type_dropdown = widgets.Dropdown(
    options=['Bar Chart', 'Line Plot', 'Scatter Plot'],
    value='Bar Chart',
    description='Chart Type:'
)

# Observe changes in dropdowns
x_dropdown.observe(update_plot, names='value')
y_dropdown.observe(update_plot, names='value')
identifier_dropdown.observe(update_plot, names='value')
chart_type_dropdown.observe(update_plot, names='value')

# Observe changes in Y-axis dropdown to update Identifier
y_dropdown.observe(on_y_axis_change, names='value')

# If sheet is changed
def on_sheet_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        load_sheet(change['new'])

sheet_dropdown.observe(on_sheet_change)

# Initial load of the selected sheet
load_sheet(sheet_dropdown.value)

# Arrange widgets in the layout
controls_box = VBox([x_dropdown, y_dropdown, identifier_dropdown, chart_type_dropdown])

# Display the outputs
display(output_preview)
display(controls_box)
display(output_plot)